In [ ]:
# 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import requests
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from matplotlib import rc

In [ ]:
외국인관광도시민박업_2022 = pd.read_csv('/content/drive/MyDrive/부산 관광 아이디어 공모전/데이터/스코어링용 데이터/숙박시설개수 데이터/fulldata_03_11_04_P_외국인관광도시민박업.csv', encoding='cp949')
일반야영장업_2022 = pd.read_csv('/content/drive/MyDrive/부산 관광 아이디어 공모전/데이터/스코어링용 데이터/숙박시설개수 데이터/fulldata_03_11_07_P_일반야영장업.csv', encoding='cp949')
한옥체험업_2022 = pd.read_csv('/content/drive/MyDrive/부산 관광 아이디어 공모전/데이터/스코어링용 데이터/숙박시설개수 데이터/fulldata_03_11_06_P_한옥체험업.csv', encoding='cp949')
관광펜션업_2022 = pd.read_csv('/content/drive/MyDrive/부산 관광 아이디어 공모전/데이터/스코어링용 데이터/숙박시설개수 데이터/fulldata_03_11_02_P_관광펜션업.csv', encoding='cp949')
관광숙박업_2022 = pd.read_csv('/content/drive/MyDrive/부산 관광 아이디어 공모전/데이터/스코어링용 데이터/숙박시설개수 데이터/fulldata_03_11_01_P_관광숙박업.csv', encoding='cp949')

# 숙박유형(숙박시설개수)

## 전처리

In [ ]:
# 영업/정상 업소만 사용
외국인관광도시민박업_2022 = 외국인관광도시민박업_2022[외국인관광도시민박업_2022.영업상태명 == '영업/정상'].reset_index(drop=True)
일반야영장업_2022 = 일반야영장업_2022[일반야영장업_2022.영업상태명 == '영업/정상'].reset_index(drop=True)
한옥체험업_2022 = 한옥체험업_2022[한옥체험업_2022.영업상태명 == '영업/정상'].reset_index(drop=True)
관광펜션업_2022 = 관광펜션업_2022[관광펜션업_2022.영업상태명 == '영업/정상'].reset_index(drop=True)
관광숙박업_2022 = 관광숙박업_2022[관광숙박업_2022.영업상태명 == '영업/정상'].reset_index(drop=True)

In [ ]:
# 필요 없는 열 제거
외국인관광도시민박업_2022 = 외국인관광도시민박업_2022[['사업장명', '소재지전체주소', '도로명전체주소', '도로명우편번호']]
일반야영장업_2022 = 일반야영장업_2022[['사업장명', '소재지전체주소', '도로명전체주소', '도로명우편번호']]
한옥체험업_2022 = 한옥체험업_2022[['사업장명', '소재지전체주소', '도로명전체주소', '도로명우편번호']]
관광펜션업_2022 = 관광펜션업_2022[['사업장명', '소재지전체주소', '도로명전체주소', '도로명우편번호']]
관광숙박업_2022 = 관광숙박업_2022[['사업장명', '소재지전체주소', '도로명전체주소', '도로명우편번호']]

In [ ]:
# 부산 소재 업소들만 추출
외국인관광도시민박업_2022 = 외국인관광도시민박업_2022[외국인관광도시민박업_2022.소재지전체주소.str.contains('부산')].reset_index(drop=True)
일반야영장업_2022 = 일반야영장업_2022[일반야영장업_2022.소재지전체주소.str.contains('부산')].reset_index(drop=True)
한옥체험업_2022 = 한옥체험업_2022[한옥체험업_2022.소재지전체주소.str.contains('부산')].reset_index(drop=True)
관광펜션업_2022 = 관광펜션업_2022[관광펜션업_2022.소재지전체주소.str.contains('부산')].reset_index(drop=True)
관광숙박업_2022 = 관광숙박업_2022[관광숙박업_2022.소재지전체주소.str.contains('부산')].reset_index(drop=True)

In [ ]:
# 소재지전체주소 => 행정동 추출
addrs = 외국인관광도시민박업_2022.소재지전체주소

locations = []
for addr in tqdm(addrs):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addr)
    headers = {
    ## 여러분의 카카오 API의 REST API키를 아래 예시와 같이 입력해주세요
    ## "Authorization": "KakaoAK REST API키 입력 gogo"}
    "Authorization": "KakaoAK 03b8b54d311c32d17d2e8abcd2ef33ab"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)

city = [] ## 시, 군
town = [] ## 동, 읍, 면
for i in range(len(locations)):

    try:
        city.append(locations[i][0].get('address').get('region_2depth_name'))
        town.append(locations[i][0].get('address').get('region_3depth_h_name'))

    except IndexError:
        print(i,'번째 주소 못가져옴', end ='')
        print()
        city.append('없음')
        town.append('없음')

    except AttributeError:
        city.append(locations[i][0].get('road_address').get('region_2depth_name'))
        town.append(locations[i][0].get('road_address').get('region_3depth_h_name'))

city_town = np.array([city,town]).T
외국인관광도시민박업_2022_temp = pd.DataFrame(city_town, columns = ['region_2depth_name','region_3depth_h_name'])
외국인관광도시민박업_2022_temp

100%|██████████| 144/144 [02:40<00:00,  1.11s/it]


,region_2depth_name,region_3depth_h_name
0,동구,초량6동
1,영도구,동삼1동
2,수영구,광안4동
3,서구,서대신4동
4,수영구,광안2동
...,...,...
139,영도구,청학2동
140,해운대구,송정동
141,영도구,청학1동
142,오산시,중앙동


In [ ]:
외국인관광도시민박업_2022['행정동명'] = 외국인관광도시민박업_2022_temp.region_3depth_h_name

In [ ]:
외국인관광도시민박업_2022[외국인관광도시민박업_2022['행정동명'] == '']

,사업장명,소재지전체주소,도로명전체주소,도로명우편번호,행정동명
46,해피하우스,부산광역시 해운대구 반여동,"부산광역시 해운대구 반여로 133, 103동 7층 701호 (반여동, 장산1차롯데낙...",48035.0,
50,센텀 게스트하우스,부산광역시 해운대구 반여동,"부산광역시 해운대구 삼어로 55, 109동 1층 101호 (반여동, 센텀에스케이뷰아파트)",48046.0,
51,해운대 몽돌 하우스,부산광역시 해운대구 좌동,"부산광역시 해운대구 좌동순환로99번길 22, 104동 9층 901호 (좌동, 경남아...",48080.0,
52,홈 스위트 홈,부산광역시 해운대구 좌동,"부산광역시 해운대구 대천로103번길 29, 504동 7층 704호 (좌동, 대동타운)",48078.0,
53,리얼 센텀하우스,부산광역시 해운대구 재송동,"부산광역시 해운대구 센텀중앙로 145, 102동 1102호 (재송동, 더샵센텀파크1...",48050.0,
54,마더 리스 모닝캄,부산광역시 해운대구 우동,"부산광역시 해운대구 해운대로 284, 107동 2204호 (우동, 센텀센시빌아파트)",48059.0,
55,비치 하우스,부산광역시 해운대구 중동,부산광역시 해운대구 달맞이길62번가길 25 (중동),48098.0,
56,메디테이션 하우스,부산광역시 해운대구 좌동,"부산광역시 해운대구 양운로 182, 104동 703호 (좌동, 두산1차아파트)",48075.0,
58,타이완 마미 하우스,부산광역시 해운대구 좌동,"부산광역시 해운대구 대천로103번길 47, 101동 2504호 (좌동, 대림1차아파트)",48075.0,
59,온앤온 하우스,부산광역시 해운대구 좌동,"부산광역시 해운대구 세실로 174, 108동 2202호 (좌동, 삼성아파트)",48076.0,


In [ ]:
외국인관광도시민박업_2022.loc[외국인관광도시민박업_2022['행정동명'] == '', '행정동명'] = ['반여1동', '반여4동', '좌3동', '좌1동', '재송1동',
                                                              '우2동', '중1동', '좌4동', '좌4동', '좌4동', '중1동', '중1동', '우3동', '우3동',
                                                              '우2동', '서대신1동']

In [ ]:
# 소재지전체주소 => 행정동 추출
addrs = 일반야영장업_2022.소재지전체주소

locations = []
for addr in tqdm(addrs):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addr)
    headers = {
    ## 여러분의 카카오 API의 REST API키를 아래 예시와 같이 입력해주세요
    ## "Authorization": "KakaoAK REST API키 입력 gogo"}
    "Authorization": "KakaoAK 03b8b54d311c32d17d2e8abcd2ef33ab"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)

city = [] ## 시, 군
town = [] ## 동, 읍, 면
for i in range(len(locations)):

    try:
        city.append(locations[i][0].get('address').get('region_2depth_name'))
        town.append(locations[i][0].get('address').get('region_3depth_h_name'))

    except IndexError:
        print(i,'번째 주소 못가져옴', end ='')
        print()
        city.append('없음')
        town.append('없음')

    except AttributeError:
        city.append(locations[i][0].get('road_address').get('region_2depth_name'))
        town.append(locations[i][0].get('road_address').get('region_3depth_h_name'))

city_town = np.array([city,town]).T
일반야영장업_2022_temp = pd.DataFrame(city_town, columns = ['region_2depth_name','region_3depth_h_name'])
일반야영장업_2022_temp

100%|██████████| 11/11 [00:12<00:00,  1.10s/it]

2 번째 주소 못가져옴


,region_2depth_name,region_3depth_h_name
0,동구,초량3동
1,영도구,청학1동
2,없음,없음
3,기장군,일광읍
4,기장군,일광읍
5,기장군,일광읍
6,기장군,장안읍
7,기장군,정관읍
8,기장군,장안읍
9,사상구,삼락동


In [ ]:
일반야영장업_2022['행정동명'] = 일반야영장업_2022_temp.region_3depth_h_name

In [ ]:
일반야영장업_2022[일반야영장업_2022.행정동명 == '없음']

,사업장명,소재지전체주소,도로명전체주소,도로명우편번호,행정동명
2,만석꾼캠핑장,"부산광역시 기장군 장안읍 월내리 368-1,368-2",NaN,NaN,없음


In [ ]:
일반야영장업_2022.loc[일반야영장업_2022.행정동명 == '없음', '행정동명'] = '장안읍'

In [ ]:
# 소재지전체주소 => 행정동 추출
addrs = 한옥체험업_2022.소재지전체주소

locations = []
for addr in tqdm(addrs):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addr)
    headers = {
    ## 여러분의 카카오 API의 REST API키를 아래 예시와 같이 입력해주세요
    ## "Authorization": "KakaoAK REST API키 입력 gogo"}
    "Authorization": "KakaoAK 03b8b54d311c32d17d2e8abcd2ef33ab"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)

city = [] ## 시, 군
town = [] ## 동, 읍, 면
for i in range(len(locations)):

    try:
        city.append(locations[i][0].get('address').get('region_2depth_name'))
        town.append(locations[i][0].get('address').get('region_3depth_h_name'))

    except IndexError:
        print(i,'번째 주소 못가져옴', end ='')
        print()
        city.append('없음')
        town.append('없음')

    except AttributeError:
        city.append(locations[i][0].get('road_address').get('region_2depth_name'))
        town.append(locations[i][0].get('road_address').get('region_3depth_h_name'))

city_town = np.array([city,town]).T
한옥체험업_2022_temp = pd.DataFrame(city_town, columns = ['region_2depth_name','region_3depth_h_name'])
한옥체험업_2022_temp

100%|██████████| 4/4 [00:04<00:00,  1.10s/it]


,region_2depth_name,region_3depth_h_name
0,기장군,철마면
1,기장군,정관읍
2,기장군,일광읍
3,기장군,장안읍


In [ ]:
한옥체험업_2022['행정동명'] = 한옥체험업_2022_temp.region_3depth_h_name

In [ ]:
# 소재지전체주소 => 행정동 추출
addrs = 관광펜션업_2022.소재지전체주소

locations = []
for addr in tqdm(addrs):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addr)
    headers = {
    ## 여러분의 카카오 API의 REST API키를 아래 예시와 같이 입력해주세요
    ## "Authorization": "KakaoAK REST API키 입력 gogo"}
    "Authorization": "KakaoAK 03b8b54d311c32d17d2e8abcd2ef33ab"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)

city = [] ## 시, 군
town = [] ## 동, 읍, 면
for i in range(len(locations)):

    try:
        city.append(locations[i][0].get('address').get('region_2depth_name'))
        town.append(locations[i][0].get('address').get('region_3depth_h_name'))

    except IndexError:
        print(i,'번째 주소 못가져옴', end ='')
        print()
        city.append('없음')
        town.append('없음')

    except AttributeError:
        city.append(locations[i][0].get('road_address').get('region_2depth_name'))
        town.append(locations[i][0].get('road_address').get('region_3depth_h_name'))

city_town = np.array([city,town]).T
관광펜션업_2022_temp = pd.DataFrame(city_town, columns = ['region_2depth_name','region_3depth_h_name'])
관광펜션업_2022_temp

100%|██████████| 1/1 [00:01<00:00,  1.11s/it]


,region_2depth_name,region_3depth_h_name
0,기장군,기장읍


In [ ]:
관광펜션업_2022['행정동명'] = 관광펜션업_2022_temp.region_3depth_h_name

In [ ]:
# 소재지전체주소 => 행정동 추출
addrs = 관광숙박업_2022.소재지전체주소

locations = []
for addr in tqdm(addrs):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addr)
    headers = {
    ## 여러분의 카카오 API의 REST API키를 아래 예시와 같이 입력해주세요
    ## "Authorization": "KakaoAK REST API키 입력 gogo"}
    "Authorization": "KakaoAK 03b8b54d311c32d17d2e8abcd2ef33ab"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)

city = [] ## 시, 군
town = [] ## 동, 읍, 면
for i in range(len(locations)):

    try:
        city.append(locations[i][0].get('address').get('region_2depth_name'))
        town.append(locations[i][0].get('address').get('region_3depth_h_name'))

    except IndexError:
        print(i,'번째 주소 못가져옴', end ='')
        print()
        city.append('없음')
        town.append('없음')

    except AttributeError:
        city.append(locations[i][0].get('road_address').get('region_2depth_name'))
        town.append(locations[i][0].get('road_address').get('region_3depth_h_name'))

city_town = np.array([city,town]).T
관광숙박업_2022_temp = pd.DataFrame(city_town, columns = ['region_2depth_name','region_3depth_h_name'])
관광숙박업_2022_temp

100%|██████████| 212/212 [03:55<00:00,  1.11s/it]

32 번째 주소 못가져옴
102 번째 주소 못가져옴
136 번째 주소 못가져옴
140 번째 주소 못가져옴


,region_2depth_name,region_3depth_h_name
0,기장군,기장읍
1,수영구,민락동
2,수영구,수영동
3,수영구,민락동
4,수영구,민락동
...,...,...
207,사하구,하단2동
208,수영구,남천1동
209,수영구,민락동
210,사상구,괘법동


In [ ]:
관광숙박업_2022['행정동명'] = 관광숙박업_2022_temp.region_3depth_h_name

In [ ]:
관광숙박업_2022[관광숙박업_2022.행정동명 == '']

,사업장명,소재지전체주소,도로명전체주소,도로명우편번호,행정동명
36,3355호스텔,부산광역시 해운대구 중동,"부산광역시 해운대구 달맞이길62번가길 31, 2층 (중동)",48098.0,
37,J. SUN 게스트하우스,부산광역시 해운대구 우동,부산광역시 해운대구 우동1로20번길 5 (우동),48087.0,
38,베니키아호텔 해운대,부산광역시 해운대구 중동,"부산광역시 해운대구 해운대해변로 317 (중동, 베니키아해운대호텔)",48095.0,
72,캔버스보산주식회사,부산광역시 해운대구 우동,"부산광역시 해운대구 해운대해변로197번길 13 (우동, 아이비모텔)",48093.0,
73,토요코인 부산 해운대-Ⅱ,부산광역시 해운대구 우동,"부산광역시 해운대구 해운대해변로237번길 5 (우동, 토요코인부산해운대2호점)",48093.0,
74,이비스 앰배서더 해운대,부산광역시 해운대구 우동,부산광역시 해운대구 해운대해변로237번길 12 (우동),48093.0,


In [ ]:
관광숙박업_2022.loc[관광숙박업_2022.행정동명 == '', '행정동명'] = ['중1동', '우1동', '중1동', '우1동', '우1동', '우1동']

In [ ]:
관광숙박업_2022[관광숙박업_2022.행정동명 == '없음']

,사업장명,소재지전체주소,도로명전체주소,도로명우편번호,행정동명
32,베니키아해운대호텔마리안느(주),부산광역시 해운대구 중동 1400-57 마리안느호텔,"부산광역시 해운대구 해운대해변로 310 (중동, 마리안느호텔)",48099.0,없음
102,더뷰 호스텔,"부산광역시 수영구 광안동 199-6번지 5, 9층",부산광역시 수영구 남천바다로33번길 47 (광안동),48303.0,없음
136,앙스모멍,부산광역시 해운대구 송정동 291-11 3~5층 일부,"부산광역시 해운대구 송정중앙로36번길 25, 3~5층 (송정동)",48072.0,없음
140,누리다,부산광역시 수영구 민락동 181-1 3~5층,"부산광역시 수영구 광안해변로 282, 3~5층 (민락동)",48283.0,없음


In [ ]:
관광숙박업_2022.loc[관광숙박업_2022.행정동명 == '없음', '행정동명'] = ['중1동', '광안2동', '송정동', '민락동']

In [ ]:
행정동별_외국인관광도시민박업_2022 = 외국인관광도시민박업_2022.groupby('행정동명', as_index=False).size()
행정동별_일반야영장업_2022 = 일반야영장업_2022.groupby('행정동명', as_index=False).size()
행정동별_한옥체험업_2022 = 한옥체험업_2022.groupby('행정동명', as_index=False).size()
행정동별_관광펜션업_2022 = 관광펜션업_2022.groupby('행정동명', as_index=False).size()
행정동별_관광숙박업_2022 = 관광숙박업_2022.groupby('행정동명', as_index=False).size()

In [ ]:
df1 = pd.merge(행정동별_외국인관광도시민박업_2022, 행정동별_일반야영장업_2022, on = '행정동명', how = 'outer')
df2 = pd.merge(df1, 행정동별_한옥체험업_2022, on = '행정동명', how = 'outer')
df3 = pd.merge(df2, 행정동별_관광펜션업_2022, on = '행정동명', how = 'outer')
df4 = pd.merge(df3, 행정동별_관광숙박업_2022, on = '행정동명', how = 'outer')
df4

<ipython-input-243-837c157275c2>:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'size_x'} in the result is deprecated and will raise a MergeError in a future version.
  df3 = pd.merge(df2, 행정동별_관광펜션업_2022, on = '행정동명', how = 'outer')


,행정동명,size_x,size_y,size_x,size_y,size
0,감전동,1.0,NaN,NaN,NaN,NaN
1,감천1동,1.0,NaN,NaN,NaN,NaN
2,감천2동,5.0,NaN,NaN,NaN,NaN
3,광안1동,2.0,NaN,NaN,NaN,NaN
4,광안2동,11.0,NaN,NaN,NaN,20.0
...,...,...,...,...,...,...
82,연산5동,NaN,NaN,NaN,NaN,3.0
83,영주1동,NaN,NaN,NaN,NaN,2.0
84,온천1동,NaN,NaN,NaN,NaN,1.0
85,전포2동,NaN,NaN,NaN,NaN,1.0


In [ ]:
df4.columns = ['행정동명', '외국인관광도시민박업', '일반야영장업', '한옥체험업', '관광펜션업', '관광숙박업']
df4.fillna(0, inplace=True)
df4

,행정동명,외국인관광도시민박업,일반야영장업,한옥체험업,관광펜션업,관광숙박업
0,감전동,1.0,0.0,0.0,0.0,0.0
1,감천1동,1.0,0.0,0.0,0.0,0.0
2,감천2동,5.0,0.0,0.0,0.0,0.0
3,광안1동,2.0,0.0,0.0,0.0,0.0
4,광안2동,11.0,0.0,0.0,0.0,20.0
...,...,...,...,...,...,...
82,연산5동,0.0,0.0,0.0,0.0,3.0
83,영주1동,0.0,0.0,0.0,0.0,2.0
84,온천1동,0.0,0.0,0.0,0.0,1.0
85,전포2동,0.0,0.0,0.0,0.0,1.0


In [ ]:
df4.to_csv('부산광역시_행정동별_숙박시설.csv', index=False)

# 퇴근 후 컨텐츠(관광시설)

## 전처리

In [ ]:
명소정보_2022 = pd.read_csv('/content/drive/MyDrive/부산 관광 아이디어 공모전/데이터/스코어링용 데이터/퇴근 후 컨텐츠 데이터/관광지 명소 정보.csv')
취미여가상품_2022 = pd.read_csv('/content/drive/MyDrive/부산 관광 아이디어 공모전/데이터/스코어링용 데이터/퇴근 후 컨텐츠 데이터/지역별 취미여가 상품.csv')
스포츠_체육_레저시설_2022 = pd.read_csv('/content/drive/MyDrive/부산 관광 아이디어 공모전/데이터/스코어링용 데이터/퇴근 후 컨텐츠 데이터/국내 지역별 스포츠관람, 체육시설, 레저스포츠 시설데이터.csv')

### 명소정보

In [ ]:
명소정보_2022 = 명소정보_2022[['여행지', 'addr1']]

In [ ]:
명소정보_2022

,여행지,addr1
0,흰여울문화마을,부산광역시 영도구 흰여울길
1,깡깡이 예술마을,부산시 영도구 대평북로 36 깡깡이 안내센터
2,국립해양박물관,부산시 영도구 해양로301번길 45
3,태종대 유원지,부산광역시 영도구 전망로 24
4,죽성성당,부산광역시 기장군 기장읍 죽성리 134-7
...,...,...
122,해운대 수목원,부산광역시 해운대구 석대동 77
123,캐비네 드 쁘아송,캐비네 드 쁘아송 (아난티코브 미디어아트 갤러리)부산광역시 기장군 기장읍 기장해안로...
124,영도 봉산마을,부산광역시 영도구 하나길 788 3층(봉산마을도시재생현장지원센터)
125,광안종합시장,부산 수영구 무학로49번길 71


In [ ]:
# 소재지전체주소 => 행정동 추출
addrs = 명소정보_2022.addr1

locations = []
for addr in tqdm(addrs):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addr)
    headers = {
    ## 여러분의 카카오 API의 REST API키를 아래 예시와 같이 입력해주세요
    ## "Authorization": "KakaoAK REST API키 입력 gogo"}
    "Authorization": "KakaoAK 03b8b54d311c32d17d2e8abcd2ef33ab"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)

100%|██████████| 127/127 [01:22<00:00,  1.54it/s]


In [ ]:
city = [] ## 시, 군
town = [] ## 동, 읍, 면
for i in range(len(locations)):

    try:
        city.append(locations[i][0].get('address').get('region_2depth_name'))
        town.append(locations[i][0].get('address').get('region_3depth_h_name'))

    except IndexError:
        print(i,'번째 주소 못가져옴', end ='')
        print()
        city.append('없음')
        town.append('없음')

    except AttributeError:
        city.append(locations[i][0].get('road_address').get('region_2depth_name'))
        town.append(locations[i][0].get('road_address').get('region_3depth_h_name'))

city_town = np.array([city,town]).T
명소정보_2022_temp = pd.DataFrame(city_town, columns = ['region_2depth_name','region_3depth_h_name'])
명소정보_2022_temp

8 번째 주소 못가져옴
21 번째 주소 못가져옴
26 번째 주소 못가져옴
56 번째 주소 못가져옴
69 번째 주소 못가져옴
77 번째 주소 못가져옴
78 번째 주소 못가져옴
83 번째 주소 못가져옴
104 번째 주소 못가져옴
105 번째 주소 못가져옴
106 번째 주소 못가져옴
111 번째 주소 못가져옴
115 번째 주소 못가져옴
123 번째 주소 못가져옴


,region_2depth_name,region_3depth_h_name
0,영도구,None
1,영도구,남항동
2,영도구,동삼1동
3,영도구,동삼2동
4,기장군,기장읍
...,...,...
122,해운대구,반송1동
123,없음,없음
124,영도구,봉래2동
125,수영구,광안1동


In [ ]:
명소정보_2022['행정동명'] = 명소정보_2022_temp.region_3depth_h_name

In [ ]:
명소정보_2022[명소정보_2022.행정동명 == '']

,여행지,addr1,행정동명
7,임랑해수욕장,부산광역시 기장군 장안읍 임랑리,
84,백양산,부산광역시 부산진구 당감동,
90,가덕도 연대봉,부산광역시 강서구 천성동,


In [ ]:
명소정보_2022.loc[명소정보_2022.행정동명 == '', '행정동명'] = ['장안읍', '당감1동', '가덕도동']

In [ ]:
명소정보_2022[명소정보_2022.행정동명 == '없음']

,여행지,addr1,행정동명
8,"문화공감 수정, 초량1941",문화공감 수정 부산광역시 동구 홍곡로 75초량1941 부산광역시 동구 망양로 533-5,없음
21,"유엔기념공원, 유엔평화기념관",유엔기념공원 부산광역시 남구 유엔평화로 93 유엔평화기념관 부산광역시 남구 홍곡로3...,없음
26,스카이워크 시리즈,오륙도 스카이워크 부산광역시 남구 오륙도로 137 송도 구름산책로 부산광역시 서구 ...,없음
56,청사포와 미포,청사포 부산광역시 해운대구 청사포로128번길 25미포 부산광역시 해운대구 달맞이길...,없음
69,"중앙공원, 민주공원",중앙공원 부산광역시 서구 망양로 193번길 187민주공원 부산광역시시 중구 민주공원...,없음
77,동래읍성,"부산광역시 동래구 명륜, 복천, 칠산, 명장, 안락동 일대",없음
78,상해거리,부산 동구 중앙대로179번길 11,없음
83,장안사,부산광역시 기장군 장안읍 장안로 482,없음
104,부산시티투어버스 BUTI,-,없음
105,황령산전망쉼터,-,없음


In [ ]:
# 104행의 시티투어버스는 제외
임시_df = pd.DataFrame({'여행지' : ['문화공감 수정', '초량1941', '유엔기념공원', '유엔평화기념관', '오륙도 스카이워크', '송도 구름산책로',
                       '청사포', '미포', '중앙공원', '민주공원', '동래읍성', '상해거리', '장안사', '황령산전망쉼터', '부산타워', '부산치유의 숲',
                       '캐비네 드 쁘아송'],
              'addr1' : list(range(17)),
              '행정동명' : ['수정1동', '초량6동', '대연4동', '대연4동', '용호2동', '암남동', '중2동', '중1동', '영주2동', '영주2동', '명륜동',
                        '초량1동', '장안읍', '대연3동', '광복동', '철마면', '기장읍']})
임시_df

,여행지,addr1,행정동명
0,문화공감 수정,0,수정1동
1,초량1941,1,초량6동
2,유엔기념공원,2,대연4동
3,유엔평화기념관,3,대연4동
4,오륙도 스카이워크,4,용호2동
5,송도 구름산책로,5,암남동
6,청사포,6,중2동
7,미포,7,중1동
8,중앙공원,8,영주2동
9,민주공원,9,영주2동


In [ ]:
# 해당 오류 행들 제거
명소정보_2022 = 명소정보_2022.drop([8,21,26,56,69,77,78,83,104,105,106,111,115,123], axis=0)
명소정보_2022

,여행지,addr1,행정동명
0,흰여울문화마을,부산광역시 영도구 흰여울길,None
1,깡깡이 예술마을,부산시 영도구 대평북로 36 깡깡이 안내센터,남항동
2,국립해양박물관,부산시 영도구 해양로301번길 45,동삼1동
3,태종대 유원지,부산광역시 영도구 전망로 24,동삼2동
4,죽성성당,부산광역시 기장군 기장읍 죽성리 134-7,기장읍
...,...,...,...
121,부산칠드런스뮤지엄,부산광역시 기장군 기장읍 기장해안로 172 스타테라스 2F,기장읍
122,해운대 수목원,부산광역시 해운대구 석대동 77,반송1동
124,영도 봉산마을,부산광역시 영도구 하나길 788 3층(봉산마을도시재생현장지원센터),봉래2동
125,광안종합시장,부산 수영구 무학로49번길 71,광안1동


In [ ]:
# 오류 수정 데이터 부착
명소정보_2022 = pd.concat([명소정보_2022, 임시_df], axis=0).reset_index(drop=True)
명소정보_2022

,여행지,addr1,행정동명
0,흰여울문화마을,부산광역시 영도구 흰여울길,None
1,깡깡이 예술마을,부산시 영도구 대평북로 36 깡깡이 안내센터,남항동
2,국립해양박물관,부산시 영도구 해양로301번길 45,동삼1동
3,태종대 유원지,부산광역시 영도구 전망로 24,동삼2동
4,죽성성당,부산광역시 기장군 기장읍 죽성리 134-7,기장읍
...,...,...,...
125,장안사,12,장안읍
126,황령산전망쉼터,13,대연3동
127,부산타워,14,광복동
128,부산치유의 숲,15,철마면


In [ ]:
명소정보_2022.행정동명.tolist() # 0, 80 행에 None 존재

[None,
 '남항동',
 '동삼1동',
 '동삼2동',
 '기장읍',
 '철마면',
 '기장읍',
 '장안읍',
 '일광읍',
 '초량2동',
 '동광동',
 '동광동',
 '범일2동',
 '온천1동',
 '대청동',
 '보수동',
 '부민동',
 '부민동',
 '대연4동',
 '대연4동',
 '용호2동',
 '용호4동',
 '송정동',
 '기장읍',
 '민락동',
 '암남동',
 '다대1동',
 '아미동',
 '우2동',
 '기장읍',
 '기장읍',
 '남산동',
 '복산동',
 '온천2동',
 '남부민1동',
 '아미동',
 '수영동',
 '대청동',
 '하단1동',
 '우2동',
 '구포2동',
 '구포2동',
 '화명3동',
 '삼락동',
 '암남동',
 '장림1동',
 '가덕도동',
 '녹산동',
 '기장읍',
 '전포2동',
 '초읍동',
 '연지동',
 '초읍동',
 '우2동',
 '부전1동',
 '범천2동',
 '신선동',
 '우3동',
 '우2동',
 '동광동',
 '감천2동',
 '다대1동',
 '동광동',
 '광복동',
 '남항동',
 '대연3동',
 '망미2동',
 '중1동',
 '광안2동',
 '금성동',
 '온천1동',
 '청룡노포동',
 '범일1동',
 '복산동',
 '우2동',
 '우2동',
 '당감1동',
 '초읍동',
 '좌4동',
 '만덕1동',
 None,
 '명륜동',
 '가덕도동',
 '당리동',
 '장안읍',
 '서대신4동',
 '화명2동',
 '부암3동',
 '모라3동',
 '선두구동',
 '우암동',
 '안락1동',
 '전포1동',
 '삼락동',
 '부전2동',
 '사직2동',
 '암남동',
 '장안읍',
 '암남동',
 '우암동',
 '아미동',
 '중1동',
 '우2동',
 '일광읍',
 '덕포2동',
 '철마면',
 '문현2동',
 '양정1동',
 '기장읍',
 '반송1동',
 '봉래2동',
 '광안1동',
 '기장읍',
 '수정1동',
 '초량6동',
 '대연4동',
 '대연4동',
 '용호2동',
 '암

In [ ]:
명소정보_2022.loc[[0, 80], :]

,여행지,addr1,행정동명
0,흰여울문화마을,부산광역시 영도구 흰여울길,None
80,회동수원지,부산광역시 금정구 수원지로,None


In [ ]:
명소정보_2022.loc[[0, 80], '행정동명'] == ['영선2동', '금사회동동']

0     False
80    False
Name: 행정동명, dtype: bool

In [ ]:
행정동별_명소정보_2022 = 명소정보_2022.groupby('행정동명', as_index=False).size()
행정동별_명소정보_2022

,행정동명,size
0,가덕도동,2
1,감천2동,1
2,광복동,2
3,광안1동,1
4,광안2동,1
...,...,...
72,초량6동,1
73,초읍동,3
74,하단1동,1
75,화명2동,1


### 취미여가상품

In [ ]:
취미여가상품_2022 = 취미여가상품_2022.loc[취미여가상품_2022.HOBBY_ND_LSR_GOODS_ADDR.str.contains('부산'), ['HOBBY_ND_LSR_GOODS_NM', 'HOBBY_ND_LSR_GOODS_ADDR']].reset_index(drop=True)

In [ ]:
취미여가상품_2022

,HOBBY_ND_LSR_GOODS_NM,HOBBY_ND_LSR_GOODS_ADDR
0,[부산] 미플 피부관리샵 롯데마트 금정점,부산 금정구 중앙대로 1731
1,[부산 남구] 포쉬네일 스파 부산본점,부산 남구 수영로 324
2,"[부산 금정] 동반할인! 내 맘대로 카드지갑, 여권지갑, 키링 만들기",부산 동래구 아시아드대로181번길 32 1층 베르데가죽공방
3,[부산 부산진구] 포쉬네일 가야점,부산 부산진구 가야대로 506
4,[부산 진구] 미플 피부관리샵 홈플러스 서면점,부산 부산진구 전포1동
5,[부산 진구] 피부관리샵 홈플러스 가야점,부산 부산진구 가야대로 506
6,[부산] 포쉬네일 덕천점,부산 북구 만덕대로 23
7,[부산 사상] 미플 피부관리샵 롯데마트 사상점,부산 사상구 낙동대로 733
8,[부산] 포쉬네일 장림점,부산 사하구 다대로 225
9,[경남 양산] 통도 환타지아 자유이용권 3월,부산 중구 대청로141번길 3


In [ ]:
# 소재지전체주소 => 행정동 추출
addrs = 취미여가상품_2022.HOBBY_ND_LSR_GOODS_ADDR

locations = []
for addr in tqdm(addrs):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={}'.format(addr)
    headers = {
    ## 여러분의 카카오 API의 REST API키를 아래 예시와 같이 입력해주세요
    ## "Authorization": "KakaoAK REST API키 입력 gogo"}
    "Authorization": "KakaoAK 03b8b54d311c32d17d2e8abcd2ef33ab"}
    place = requests.get(url, headers = headers).json()['documents']
    locations.append(place)

city = [] ## 시, 군
town = [] ## 동, 읍, 면
for i in range(len(locations)):

    try:
        city.append(locations[i][0].get('address').get('region_2depth_name'))
        town.append(locations[i][0].get('address').get('region_3depth_h_name'))

    except IndexError:
        print(i,'번째 주소 못가져옴', end ='')
        print()
        city.append('없음')
        town.append('없음')

    except AttributeError:
        city.append(locations[i][0].get('road_address').get('region_2depth_name'))
        town.append(locations[i][0].get('road_address').get('region_3depth_h_name'))

city_town = np.array([city,town]).T
취미여가상품_2022_temp = pd.DataFrame(city_town, columns = ['region_2depth_name','region_3depth_h_name'])
취미여가상품_2022_temp

100%|██████████| 16/16 [00:11<00:00,  1.37it/s]


,region_2depth_name,region_3depth_h_name
0,금정구,부곡3동
1,남구,대연3동
2,동래구,사직1동
3,부산진구,가야2동
4,부산진구,전포1동
5,부산진구,가야2동
6,북구,덕천2동
7,사상구,엄궁동
8,사하구,장림2동
9,중구,중앙동


In [ ]:
취미여가상품_2022['행정동명'] = 취미여가상품_2022_temp.region_3depth_h_name

In [ ]:
# 빈 데이터 검색으로 채워넣음
취미여가상품_2022.loc[4, '행정동명'] = '전포1동'
취미여가상품_2022.loc[14, '행정동명'] = '우1동'

In [ ]:
# 중복 상품 행 제거
취미여가상품_2022 = 취미여가상품_2022.loc[[0,1,2,3,4,5,6,7,8,9,11,12,13,14,15], :].reset_index(drop=True)

In [ ]:
행정동별_취미여가상품_2022 = 취미여가상품_2022.groupby('행정동명', as_index=False).size()

In [ ]:
행정동별_취미여가상품_2022

,행정동명,size
0,가야2동,2
1,대연3동,1
2,덕천2동,1
3,반여1동,1
4,부곡3동,1
5,사직1동,1
6,엄궁동,1
7,우1동,1
8,우3동,1
9,장림2동,1


### 스포츠/체육/레저시설

In [ ]:
# 부산인 데이터만 가져오기
스포츠_체육_레저시설_2022 = 스포츠_체육_레저시설_2022[스포츠_체육_레저시설_2022.CTPRVN_NM.str.contains('부산')].reset_index(drop=True)

In [ ]:
# 행정동코드만 추출
스포츠_체육_레저시설_2022 = 스포츠_체육_레저시설_2022[['POI_NM', 'ADSTRD_CD']]

In [ ]:
스포츠_체육_레저시설_2022

,POI_NM,ADSTRD_CD
0,부산월드컵경기장보조경기장,2647062000
1,금창체육관,2632053000
2,극동체육관,2614064000
3,대림체육관,2623074000
4,송천체육관,2620065000
...,...,...
5495,부산수산자원연구소축구장,2644054500
5496,구민운동장,2632054300
5497,부산조정경기장,2644053000
5498,부산요트경기장,2635052500


In [ ]:
부산_행정동코드 = pd.read_csv('/content/drive/MyDrive/부산 관광 아이디어 공모전/데이터/스코어링용 데이터/부산광역시_행정동코드(10자리).csv', encoding='cp949')

In [ ]:
부산_행정동코드

,행정동코드,시도명,시군구명,읍면동명,생성일자,말소일자
0,2600000000,부산광역시,NaN,NaN,19950101,NaN
1,2611000000,부산광역시,중구,NaN,19950101,NaN
2,2611051000,부산광역시,중구,중앙동,19950101,NaN
3,2611052000,부산광역시,중구,동광동,19950101,NaN
4,2611053000,부산광역시,중구,대청동,19950101,NaN
...,...,...,...,...,...,...
217,2671025000,부산광역시,기장군,기장읍,19950301,NaN
218,2671025300,부산광역시,기장군,장안읍,19950301,NaN
219,2671025600,부산광역시,기장군,정관읍,20150923,NaN
220,2671025900,부산광역시,기장군,일광읍,20220401,NaN


In [ ]:
행정동_list = []

for i in 스포츠_체육_레저시설_2022.ADSTRD_CD:
    행정동_list.append(list(부산_행정동코드.loc[부산_행정동코드.행정동코드 == i, '읍면동명'].values))

In [ ]:
스포츠_체육_레저시설_2022['행정동명'] = sum(행정동_list, [])

In [ ]:
스포츠_체육_레저시설_2022

,POI_NM,ADSTRD_CD,행정동명
0,부산월드컵경기장보조경기장,2647062000,거제제2동
1,금창체육관,2632053000,금곡동
2,극동체육관,2614064000,충무동
3,대림체육관,2623074000,개금제1동
4,송천체육관,2620065000,동삼제1동
...,...,...,...
5495,부산수산자원연구소축구장,2644054500,명지2동
5496,구민운동장,2632054300,화명제3동
5497,부산조정경기장,2644053000,강동동
5498,부산요트경기장,2635052500,우제3동


In [ ]:
스포츠_체육_레저시설_2022.행정동명 = 스포츠_체육_레저시설_2022.행정동명.str.replace('제','')
스포츠_체육_레저시설_2022.행정동명 = 스포츠_체육_레저시설_2022.행정동명.str.replace('거','거제')

In [ ]:
행정동별_스포츠_체육_레저시설_2022 = 스포츠_체육_레저시설_2022.groupby('행정동명', as_index=False).size()

In [ ]:
행정동별_스포츠_체육_레저시설_2022

,행정동명,size
0,가덕도동,1
1,가락동,4
2,가야1동,28
3,가야2동,26
4,감만1동,11
...,...,...
195,하단2동,74
196,학장동,46
197,화명1동,30
198,화명2동,19


In [ ]:
df1 = pd.merge(행정동별_명소정보_2022, 행정동별_취미여가상품_2022, on = '행정동명', how = 'outer')
df2 = pd.merge(df1, 행정동별_스포츠_체육_레저시설_2022, on = '행정동명', how = 'outer')
df2

,행정동명,size_x,size_y,size
0,가덕도동,2.0,NaN,1.0
1,감천2동,1.0,NaN,4.0
2,광복동,2.0,NaN,17.0
3,광안1동,1.0,NaN,38.0
4,광안2동,1.0,NaN,23.0
...,...,...,...,...
197,초장동,NaN,NaN,1.0
198,충무동,NaN,NaN,22.0
199,하단2동,NaN,NaN,74.0
200,학장동,NaN,NaN,46.0


In [ ]:
df2.columns = ['행정동명', '관광명소', '취미여가상품', '스포츠_체육_레저']
df2.fillna(0, inplace=True)
df2

,행정동명,관광명소,취미여가상품,스포츠_체육_레저
0,가덕도동,2.0,0.0,1.0
1,감천2동,1.0,0.0,4.0
2,광복동,2.0,0.0,17.0
3,광안1동,1.0,0.0,38.0
4,광안2동,1.0,0.0,23.0
...,...,...,...,...
197,초장동,0.0,0.0,1.0
198,충무동,0.0,0.0,22.0
199,하단2동,0.0,0.0,74.0
200,학장동,0.0,0.0,46.0


In [ ]:
df2.to_csv('부산광역시_퇴근 후 컨텐츠(명소, 취미여가상품, 스포츠_체육_레저시설).csv', index=False)

# 식음 및 부대시설

In [ ]:
편의시설_2022 = pd.read_csv('/content/drive/MyDrive/부산 관광 아이디어 공모전/데이터/클러스터링용 데이터/데이터/전처리 데이터/부산광역시_행정동별_편의시설 접근성.csv', encoding='cp949')

In [ ]:
# 식음료 편의시설 추출
식음료_편의시설_2022 = 편의시설_2022[['행정동명', '관광식당 수(식음)', '관광유흥음식점 수(식음)', '외국인유흥음식점 수(식음)', '휴게음식점 수(식음)', '음식점 수(식음)']]

In [ ]:
편의시설_2022 = pd.DataFrame({'행정동명' : 식음료_편의시설_2022.행정동명,
 '식음료_편의시설수' : 식음료_편의시설_2022.loc[:, 식음료_편의시설_2022.columns != '행정동명'].sum(axis=1)})

In [ ]:
편의시설_2022

,행정동명,식음료_편의시설수
0,중앙동,351
1,동광동,192
2,대청동,62
3,보수동,19
4,부평동,169
...,...,...
200,기장읍,1244
201,장안읍,231
202,정관읍,1095
203,일광면,497


In [ ]:
편의시설_2022.to_csv('부산광역시_식음 및 부대시설.csv', index=False)